# Robot Javelin - Numerical Verification

This notebook verifies the analytical solution for Javalin's optimal winning probability when Spears exploits the protocol.


In [1]:
import numpy as np
from scipy import integrate

# Define constants
T = (np.sqrt(5) - 1) / 2  # Nash equilibrium threshold
T_prime = 7 / 12  # Javalin's optimal counter-threshold
s1 = 1 / 2  # Spears' threshold when j < T
s2 = T + (1 - T) ** 2 / 2  # Spears' threshold when j >= T

print(f"T (Nash threshold): {T:.10f}")
print(f"T' (Javalin counter): {T_prime:.10f}")
print(f"s1 (Spears threshold for j < T): {s1:.10f}")
print(f"s2 (Spears threshold for j >= T): {s2:.10f}")
print(f"\nVerifying 0.5 < T' < T: {0.5 < T_prime < T}")
print(f"Verifying s2 > T: {s2 > T}")

T (Nash threshold): 0.6180339887
T' (Javalin counter): 0.5833333333
s1 (Spears threshold for j < T): 0.5000000000
s2 (Spears threshold for j >= T): 0.6909830056

Verifying 0.5 < T' < T: True
Verifying s2 > T: True


## Verify Indifference Condition at T'

Check that at $j = T'$, Javalin is indifferent between keeping and rethrowing.


In [2]:
# P(win | keep T')
prob_keep = 1.5 * T_prime - 0.5

# P(win | rethrow)
prob_rethrow = 3 / 8

print(f"P(win | keep T') = {prob_keep:.10f}")
print(f"P(win | rethrow) = {prob_rethrow:.10f}")
print(f"Indifference satisfied: {np.isclose(prob_keep, prob_rethrow)}")

P(win | keep T') = 0.3750000000
P(win | rethrow) = 0.3750000000
Indifference satisfied: True


## Region 1: $j \in [0, T')$

Javalin rethrows, Spears uses threshold $s_1 = 1/2$.


In [3]:
# Region 1: Constant win probability 3/8
region1 = (3 / 8) * T_prime

print(f"Region 1 contribution: {region1:.10f}")
print(f"Expected (7/32): {7 / 32:.10f}")
print(f"Match: {np.isclose(region1, 7 / 32)}")

Region 1 contribution: 0.2187500000
Expected (7/32): 0.2187500000
Match: True


## Region 2: $j \in [T', T)$

Javalin keeps $j$, Spears uses threshold $s_1 = 1/2$.


In [4]:
# Region 2: Integrate (3/2*j - 1/2) from T' to T
def integrand_region2(j):
    return 3 / 2 * j - 1 / 2


region2, _ = integrate.quad(integrand_region2, T_prime, T)

# Also compute analytically
region2_analytical = (3 / 4 * T**2 - 1 / 2 * T) - (3 / 4 * T_prime**2 - 1 / 2 * T_prime)

print(f"Region 2 contribution (numerical): {region2:.10f}")
print(f"Region 2 contribution (analytical): {region2_analytical:.10f}")
print(f"Expected: 3/4 T² - 1/2 T + 7/192 = {3 / 4 * T**2 - 1 / 2 * T + 7 / 192:.10f}")

Region 2 contribution (numerical): 0.0139158474
Region 2 contribution (analytical): 0.0139158474
Expected: 3/4 T² - 1/2 T + 7/192 = 0.0139158474


## Region 3: $j \in [T, 1]$

Javalin keeps $j$, Spears uses threshold $s_2 = T + (1-T)^2/2$.

Split into two subregions based on whether $j < s_2$ or $j \geq s_2$.


In [5]:
# Region 3a: j in [T, s2), P(win|j) = s2 * j
def integrand_region3a(j):
    return s2 * j


region3a, _ = integrate.quad(integrand_region3a, T, s2)


# Region 3b: j in [s2, 1], P(win|j) = j(s2 + 1) - s2
def integrand_region3b(j):
    return j * (s2 + 1) - s2


region3b, _ = integrate.quad(integrand_region3b, s2, 1)

region3 = region3a + region3b

print(f"Region 3a contribution: {region3a:.10f}")
print(f"Region 3b contribution: {region3b:.10f}")
print(f"Region 3 total (numerical): {region3:.10f}")

Region 3a contribution: 0.0329915028
Region 3b contribution: 0.2282797402
Region 3 total (numerical): 0.2612712430


## Final Winning Probability

Sum all three regions to get Javalin's total probability of winning.


In [6]:
# Total probability (numerical integration)
prob_win_numerical = region1 + region2 + region3

print(f"\n{'=' * 70}")
print("FINAL ANSWER (from numerical integration)")
print(f"{'=' * 70}")
print(f"P(Javalin wins): {prob_win_numerical:.10f}")
print(f"\nSanity check: P(win) < 0.5? {prob_win_numerical < 0.5}")
print(f"(Despite optimal counter-play, Spears' exploit gives them an edge)")
print(f"{'=' * 70}")


FINAL ANSWER (from numerical integration)
P(Javalin wins): 0.4939370904

Sanity check: P(win) < 0.5? True
(Despite optimal counter-play, Spears' exploit gives them an edge)


## Breakdown Summary


In [7]:
print(f"Region 1: {region1:.10f} ({region1 / prob_win_numerical * 100:.2f}%)")
print(f"Region 2: {region2:.10f} ({region2 / prob_win_numerical * 100:.2f}%)")
print(f"Region 3: {region3:.10f} ({region3 / prob_win_numerical * 100:.2f}%)")
print(f"Total:    {prob_win_numerical:.10f}")

Region 1: 0.2187500000 (44.29%)
Region 2: 0.0139158474 (2.82%)
Region 3: 0.2612712430 (52.90%)
Total:    0.4939370904


## Exact Symbolic Formula

Derive the simplified formula and compute with high precision.


In [8]:
from decimal import Decimal, getcontext

# Set high precision
getcontext().prec = 50

# Compute T with high precision
sqrt5 = Decimal(5).sqrt()
T_decimal = (sqrt5 - 1) / 2

print("Simplified Formula:")
print("  P(win) = 121/192 - T/2 + T²/2 - T⁴/8")
print("  where T = (√5 - 1)/2")

print("\nCombining regions (with s₂ = 1/2 + T²/2):")
print("  Region 1: 7/32")
print("  Region 2: 3/4·T² - 1/2·T + 7/192")
print("  Region 3: 3/8 - T²/4 - T⁴/8")

# Compute using simplified formula
prob_simplified = (
    Decimal(121) / Decimal(192)
    - T_decimal / Decimal(2)
    + T_decimal**2 / Decimal(2)
    - T_decimal**4 / Decimal(8)
)

print("\n" + "=" * 70)
print("Exact Symbolic Form:")
print("  P(win) = (229 - 60√5)/192")
print("\nNumerical Evaluation:")
print(f"  {str(prob_simplified)[:52]}...")

# Round to 10 decimal places
prob_10dp = prob_simplified.quantize(Decimal("0.0000000001"))
prob_11dp = prob_simplified.quantize(Decimal("0.00000000001"))

print(f"\nRounded to 10 decimal places: {prob_10dp}")
print(f"  (11th digit = {str(prob_11dp)[-1]}, rounds up)")

print("\n" + "=" * 70)
print(f"FINAL ANSWER: (229 - 60√5)/192 = {prob_10dp}")
print("=" * 70)

Simplified Formula:
  P(win) = 121/192 - T/2 + T²/2 - T⁴/8
  where T = (√5 - 1)/2

Combining regions (with s₂ = 1/2 + T²/2):
  Region 1: 7/32
  Region 2: 3/4·T² - 1/2·T + 7/192
  Region 3: 3/8 - T²/4 - T⁴/8

Exact Symbolic Form:
  P(win) = (229 - 60√5)/192

Numerical Evaluation:
  0.49393709036464905320546656185480950975814009595472...

Rounded to 10 decimal places: 0.4939370904
  (11th digit = 6, rounds up)

FINAL ANSWER: (229 - 60√5)/192 = 0.4939370904


## Monte Carlo Simulation Verification

Simulate the actual game to verify our analytical result.


In [9]:
def simulate_game(n_trials=1000000, seed=42):
    """
    Simulate the actual game to verify analytical result.
    - Javalin uses threshold T' = 7/12
    - Spears uses exploit with d = T and thresholds s1, s2
    """
    np.random.seed(seed)
    wins = 0

    for _ in range(n_trials):
        j = np.random.rand()
        s = np.random.rand()

        # Javalin's strategy: rethrow if j < T'
        j_final = np.random.rand() if j < T_prime else j

        # Spears' strategy using exploit (bit: is j >= T?)
        if j >= T:
            s_final = np.random.rand() if s < s2 else s
        else:
            s_final = np.random.rand() if s < s1 else s

        if j_final > s_final:
            wins += 1

    return wins / n_trials


# Run simulation
print("Running Monte Carlo simulation...")
prob_sim = simulate_game(n_trials=1000000)

print(f"\nSimulation result (n=1,000,000): {prob_sim:.10f}")
print(f"Analytical result:                {prob_win_numerical:.10f}")
print(f"Difference:                       {abs(prob_sim - prob_win_numerical):.10f}")

Running Monte Carlo simulation...

Simulation result (n=1,000,000): 0.4940560000
Analytical result:                0.4939370904
Difference:                       0.0001189096
